## `PioModelInitializer` Class
Must implement `initialize_model()` method.

In [ ]:
class PioModelInitializer(object):
    def __init__(self, 
                 *args,
                 **kwargs):        

        pass

    
    def initialize_model(self,
                        *args,
                        **kwargs):

        return

## `PioRequestTransformer` Class
Must implement `transform_request()` method.

In [ ]:
class PioRequestTransformer(object):
    def __init__(self, 
                 *args,
                 **kwargs):        

        pass

    
    def transform_request(self,
                          request,
                          *args,
                          **kwargs):

        return request

## `PioResponseTransformer` Class
Must have `transform_response()` method.

In [ ]:
class PioResponseTransformer(object):
    def __init__(self, 
                 *args,
                 **kwargs):        

        pass

    
    def transform_response(self,
                           response,
                           *args,
                           **kwargs):

        return response

## `PioModel` Class
Must implement the `predict()` method.

In [ ]:
class PioModel(object):

    def __init__(self, 
                 request_transformer, 
                 response_transformer,
                 model_initializer,
                 *args,
                 **kwargs):

        self.request_transformer = request_transformer
        self.response_transformer = response_transformer

        self.model_initializer = model_initializer
        self.model = self.model_initializer.initialize_model(args,
                                                             kwargs)

        
    def predict(self, 
                request,
                *args,
                **kwargs):

        return

## `MyRequestTransformer` Class
Extends `PioRequestTransformer` class.  Must implement the `transform_request()` method.

### Convert `json` -> `dict`

In [ ]:
class MyRequestTransformer(PioRequestTransformer): 

    def __init__(self, 
                 *args,
                 **kwargs):
        PioRequestTransformer.__init__(self, 
                                       args, 
                                       kwargs)

        
    def transform_request(self,
                          request,
                          *args,
                          **kwargs):
        import ujson
        import numpy as np
        PioRequestTransformer.transform_request(self,
                                                request,
                                                args,
                                                kwargs)
        print(request)
        request_str = request.decode('utf-8')
        request_str = request_str.strip().replace('\n', ',')
        # surround the json with '[' ']' to prepare for conversion
        request_str = '[%s]' % request_str
        request_json = ujson.loads(request_str)
        request_transformed = ([self.parse_json_line(json_line) for json_line in request_json])
        print(request_transformed)
        return np.array(request_transformed)


    def parse_json_line(self,
                        json_line):
        # Note:  We're only using 1 feature in this demo
        return json_line['feature0']

## `MyResponseTransformer` Class
Extends `PioResponseTransformer` class.  Must implement the `transform_response()` method.

### Convert `dict` -> `json`

In [ ]:
class MyResponseTransformer(PioResponseTransformer):
    def __init__(self, 
                 *args,
                 **kwargs):

        PioResponseTransformer.__init__(self, 
                                        args, 
                                        kwargs)

        
    def transform_response(self,
                           response,
                           *args,
                           **kwargs):

        import ujson
        # input: numpy array
        # output: list of json
        transformed_response = PioResponseTransformer.transform_response(self,
                                                                         response,
                                                                         args,
                                                                         kwargs)      
        print('Transformed Response:')
        print(transformed_response)
        return ujson.dumps(transformed_response.tolist())

## Train My Model

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import datasets

# Load the diabetes dataset
diabetes = datasets.load_diabetes()

# ONLY USING 1 FEATURE FOR THIS EXAMPLE!
# Use only one feature
diabetes_X = diabetes.data[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes.target[:-20]
diabetes_y_test = diabetes.target[-20:]

# Create linear regression model
model = linear_model.LinearRegression()

# Train the model using the training sets
model.fit(diabetes_X_train, diabetes_y_train)

## Save/Pickle Underlying Model as `model.pkl`

In [ ]:
import dill as pickle

model_pkl_path = 'model.pkl'

with open(model_pkl_path, 'wb') as fh:
    pickle.dump(model, fh)

In [ ]:
import subprocess

output = subprocess.check_output('ls -l model.pkl',
                                 stderr=subprocess.STDOUT,
                                 shell=True)
print(output.decode('utf-8'))

## Create Model Initializer
Must implement the `initialize_model()` method

In [ ]:
class MyModelInitializer(PioModelInitializer): 

    def __init__(self, 
                 *args,
                 **kwargs):

        PioModelInitializer.__init__(self, 
                                     args, 
                                     kwargs)

        
    def initialize_model(self,                        
                         *args,
                         **kwargs):

        PioModelInitializer.initialize_model(self, 
                                             args, 
                                             kwargs)

        import dill as pickle
        model_pkl_path = './model.pkl'

        with open(model_pkl_path, 'rb') as fh:
            model = pickle.load(fh)
        
        return model            

## `MyModel` Class
Extends `PioModel` class.  Must implement `predict()` method.

In [ ]:
class MyModel(PioModel):

    
    def __init__(self, 
                 request_transformer, 
                 response_transformer,
                 model_initializer,
                 *args,
                 **kwargs):

        PioModel.__init__(self,
                          request_transformer,
                          response_transformer,
                          model_initializer,
                          args,
                          kwargs)
        
    def predict(self, 
                request,
                *args,
                **kwargs):

        transformed_request = self.request_transformer.transform_request(request)
        response = self.model.predict(transformed_request)
        transformed_response = self.response_transformer.transform_response(response)
        return transformed_response

## Construct `MyModel`
Inject Model with Request Transformer, Response Transformer, and Model Initializer
```
MyModel(`MyRequestTransformer`, `MyResponseTransformer`, `MyModelInitializer`)
```

In [ ]:
pio_model = MyModel(MyRequestTransformer(),
                    MyResponseTransformer(), 
                    MyModelInitializer())

## Save/Pickle Pio Model as `pio_model.pkl`

In [ ]:
import dill as pickle

pio_model_pkl_path = 'pio_model.pkl'

with open(pio_model_pkl_path, 'wb') as fh:
    pickle.dump(pio_model, fh)

In [ ]:
import subprocess

output = subprocess.check_output('ls -l pio_model.pkl',
                                 stderr=subprocess.STDOUT,
                                 shell=True)
print(output.decode('utf-8'))

In [ ]:
pio_model_pkl_path='pio_model.pkl'
test_request_path='data/test_request.json'
test_response_path='data/test_response.json'

In [ ]:
from __future__ import print_function, absolute_import, division

import json
import importlib
import dill as pickle

def test(pio_model_pkl_path, test_request_path, test_response_path):
    with open(pio_model_pkl_path, 'rb') as fh:
        pio_model = pickle.load(fh)
    with open(test_request_path, 'rb') as fh:
        actual_request = fh.read() 
    with open(test_response_path, 'rb') as fh:
        expected_response = fh.read()
    print('Expected Response:')
    print(expected_response)
    print('')
    print('Actual Request:')
    print(actual_request)
    actual_response = pio_model.predict(actual_request)
    print('')
    print('Actual Response:')
    print(actual_response)

    return (json.loads(expected_response.decode('utf-8').strip()) \
            == json.loads(actual_response.strip()))

In [ ]:
test_success = test(pio_model_pkl_path, 
                    test_request_path,
                    test_response_path)

print('')
print('Test Success: %s' % test_success)